# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
# Your code here
import sqlite3
import pandas as pd
con = sqlite3.connect('data.sqlite')
cur = con.cursor()

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [45]:
# Your code here
strSQL1 = """
    SELECT e.lastName, e.firstName, o.city, o.state
    FROM employees AS e
    JOIN offices AS o
    ON e.officeCode = o.officeCode
    WHERE o.state = 'MA' AND o.city = 'Boston'
"""
strSQL2 = """
    SELECT e.lastName, e.firstName, o.city, o.state
    FROM employees AS e
    JOIN offices AS o
        USING(officeCode)
    WHERE o.state = 'MA' AND o.city = 'Boston'
"""
pd.read_sql(strSQL2, con)

,lastName,firstName,city,state
0,Firrelli,Julie,Boston,MA
1,Patterson,Steve,Boston,MA


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [50]:
# Your code here
strSQL1 = """
    SELECT 
        o.officeCode,
        o.city, 
        o.country, 
        COUNT(e.employeeNumber) AS employeeCount
    FROM offices AS o
    LEFT JOIN employees AS e
    ON e.officeCode = o.officeCode
    GROUP BY o.officeCode
    HAVING COUNT(e.employeeNumber) < 1
"""
strSQL2 = """
    SELECT 
        o.officeCode,
        o.city, 
        o.country, 
        COUNT(e.employeeNumber) AS employeeCount
    FROM offices AS o
    LEFT JOIN employees AS e
        USING(officeCode)
    GROUP BY o.officeCode
    HAVING COUNT(e.employeeNumber) < 1
"""
pd.read_sql(strSQL2, con)

,officeCode,city,country,employeeCount
0,27,Boston,USA,0


## Write 3 questions of your own and answer them

In [55]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
strSQL = """
    SELECT 
        o.officeCode,
        o.city, 
        o.country, 
        COUNT(c.customerNumber) AS customerCount
    FROM 
        offices AS o
    JOIN 
        employees AS e
    ON 
        e.officeCode = o.officeCode
    JOIN 
        customers AS c
    ON 
        e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY o.officeCode
"""
pd.read_sql(strSQL, con)

,officeCode,city,country,customerCount
0,1,San Francisco,USA,12
1,2,Boston,USA,12
2,3,NYC,USA,15
3,4,Paris,France,29
4,5,Tokyo,Japan,5
5,6,Sydney,Australia,10
6,7,London,UK,17


In [57]:
"""
Question 1
Number of orders per office
"""

# Your code here
strSQL = """
    SELECT 
        o.officeCode,
        o.city, 
        o.country, 
        COUNT(orders.orderNumber) AS orderCount
    FROM 
        offices AS o
    LEFT JOIN 
        employees AS e
    ON 
        o.officeCode = e.officeCode
    LEFT JOIN 
        customers AS c
    ON 
        e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN
        orders
    ON
        c.customerNumber = orders.customerNumber
    GROUP BY 
        o.officeCode,
        o.city,
        o.country
    ORDER BY
        COUNT(orders.orderNumber) DESC
"""
pd.read_sql(strSQL, con)

,officeCode,city,country,orderCount
0,4,Paris,France,106
1,1,San Francisco,USA,48
2,7,London,UK,47
3,3,NYC,USA,39
4,6,Sydney,Australia,38
5,2,Boston,USA,32
6,5,Tokyo,Japan,16
7,27,Boston,USA,0


In [59]:
"""
Question 2
Items sold per office
"""
strSQL = """
    SELECT 
        o.officeCode,
        o.city, 
        o.country, 
        SUM(CAST(od.quantityOrdered AS INT)) AS totalItemsOrdered
    FROM 
        offices AS o
    LEFT JOIN 
        employees AS e
    ON 
        o.officeCode = e.officeCode
    LEFT JOIN 
        customers AS c
    ON 
        e.employeeNumber = c.salesRepEmployeeNumber
    LEFT JOIN
        orders
    ON
        c.customerNumber = orders.customerNumber
    LEFT JOIN
        orderdetails AS od
    ON
        orders.orderNumber = od.orderNumber
    GROUP BY 
        o.officeCode,
        o.city,
        o.country
    ORDER BY
        COUNT(orders.orderNumber) DESC
"""
pd.read_sql(strSQL, con)
pd.read_sql(strSQL, con)

,officeCode,city,country,totalItemsOrdered
0,4,Paris,France,33887.0
1,7,London,UK,15691.0
2,1,San Francisco,USA,15910.0
3,6,Sydney,Australia,12878.0
4,3,NYC,USA,12439.0
5,2,Boston,USA,9788.0
6,5,Tokyo,Japan,4923.0
7,27,Boston,USA,NaN


In [18]:
"""
Question 3
Orders per customer
"""

# Your code here
strSQL = """
    SELECT c.customerNumber, COUNT(o.orderNumber) as orderCount
    FROM customers AS c
    JOIN orders AS o
    ON c.customerNumber = o.customerNumber
    GROUP BY c.customerNumber
    ORDER BY COUNT(o.orderNumber) DESC
"""
pd.read_sql(strSQL, con)

,customerNumber,orderCount
0,141,26
1,124,17
2,353,5
3,323,5
4,148,5
...,...,...
93,189,2
94,177,2
95,173,2
96,171,2


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [62]:
# Your code here
"""
employees (employeeNumber) -> customers (salesRepEmployeeNumber) ->
-> orders (customerNumber)
"""
strSQL = """
    SELECT
        e.firstName,
        e.lastName,
        e.employeeNumber, 
        p.productName AS productSoldByEmployee
    FROM employees AS e
    JOIN customers AS c
    ON  e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders AS o
    ON c.customerNumber = o.customerNumber
    JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
    JOIN products AS p
    ON p.productCode = od.productCode
--    GROUP BY         
--        e.employeeNumber, 
--        p.productName
"""
pd.read_sql(strSQL, con)

,firstName,lastName,employeeNumber,productSoldByEmployee
0,Leslie,Jennings,1165,1958 Setra Bus
1,Leslie,Jennings,1165,1940 Ford Pickup Truck
2,Leslie,Jennings,1165,1939 Cadillac Limousine
3,Leslie,Jennings,1165,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1165,1968 Ford Mustang
...,...,...,...,...
2991,Martin,Gerard,1702,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1702,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,1702,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1702,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [64]:
# Your code here
strSQL = """
    SELECT 
        e.employeeNumber,
        e.lastName,
        e.firstName,
        SUM(od.quantityOrdered) AS quantitySoldByEmployee
    FROM employees AS e
    JOIN customers AS c
    ON  e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders AS o
    ON c.customerNumber = o.customerNumber
    JOIN orderDetails AS od
    ON o.orderNumber = od.orderNumber
    GROUP BY 
        e.employeeNumber,
        e.lastName,
        e.firstName
    ORDER BY 
        e.lastName, 
        e.firstName
        ASC
"""
pd.read_sql(strSQL, con)

,employeeNumber,lastName,firstName,quantitySoldByEmployee
0,1337,Bondur,Loui,6186
1,1501,Bott,Larry,8205
2,1401,Castillo,Pamela,9290
3,1188,Firrelli,Julie,4227
4,1611,Fixter,Andy,6246
5,1702,Gerard,Martin,4180
6,1370,Hernandez,Gerard,14231
7,1165,Jennings,Leslie,11854
8,1504,Jones,Barry,7486
9,1612,Marsh,Peter,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [66]:
# Your code here
strSQL = """
    SELECT DISTINCT
        e.employeeNumber, 
        e.lastName,
        e.firstName,
        COUNT(p.productCode) AS productsSoldByEmployee
    FROM employees AS e
    JOIN customers AS c
    ON  e.employeeNumber = c.salesRepEmployeeNumber
    JOIN orders AS o
    ON c.customerNumber = o.customerNumber
    JOIN orderDetails AS od
    ON o.orderNumber = od.orderNumber
    JOIN products AS p
    ON p.productCode = od.productCode
    GROUP BY         
        e.employeeNumber,
        e.lastName,
        e.firstName
    HAVING COUNT(p.productName) > 200
    ORDER BY COUNT(p.productName) DESC
"""
pd.read_sql(strSQL, con)

,employeeNumber,lastName,firstName,productsSoldByEmployee
0,1370,Hernandez,Gerard,396
1,1165,Jennings,Leslie,331
2,1401,Castillo,Pamela,272
3,1501,Bott,Larry,236
4,1504,Jones,Barry,220
5,1323,Vanauf,George,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!